<a href="https://colab.research.google.com/github/satishgc227/Derby-project/blob/main/Accurate_deep_learning_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score




In [ ]:
# Load in the data
from sklearn.preprocessing import OneHotEncoder
df = pd.read_csv('/content/edited complete.csv',encoding='latin',nrows=10000)

df['race_type\xa0'].unique()
df['course_type'].unique()
df['track_condition\xa0'].unique()
df['track_id'].unique()

s=df[['race_type\xa0','course_type','track_condition\xa0','track_id']]

ohe=OneHotEncoder()
feature_array=ohe.fit_transform(df[['race_type\xa0','course_type','track_condition\xa0','track_id']]).toarray()

feature_labels=ohe.categories_
print(feature_labels)

np.hstack(feature_labels)

feature_labels=np.hstack(feature_labels)
f=pd.DataFrame(feature_array,columns=feature_labels)
pd.concat([df,f],axis=1)

df.drop(['race_date','track_id','course_type','track_condition\xa0','race_type\xa0','jockey\xa0','program_number\xa0','latitude\xa0','longitude\xa0'],axis=1,inplace=True)
df=pd.concat([df,f],axis=1)

df.isnull().any()



"""Removing the null values"""

df.replace([np.inf,-np.inf],np.nan ,inplace=True)
df.dropna(inplace=True)
df.isnull().any()

!pip install movecolumn
import movecolumn as mc
mc.MoveToLast(df,'position_at_finish\xa0')

X = df.iloc[:,:-1]
y = df.iloc[:, -1]




[array(['ALW', 'AOC', 'CLM', 'SST'], dtype=object), array(['D', 'I', 'M', 'O', 'T'], dtype=object), array(['FM ', 'FT ', 'GD ', 'MY ', 'SY '], dtype=object), array(['AQU', 'BEL', 'SAR'], dtype=object)]
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for movecolumn: filename=movecolumn-0.0.7-py3-none-any.whl size=10440 sha256=7bd6ef53c1fecbca86c6f0ef08c49886703502050fe5b27591e4aef2778e1b39
  Stored in directory: /root/.cache/pip/wheels/4c/ad/b7/671cd4a6e1464bb816aea3bacdcc933acf25e3d47946a7edcf
Successfully built movecolumn


In [ ]:
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size= 0.2, random_state = 1)
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [ ]:
df.corr()['position_at_finish\xa0']

race_number            0.442268
trakus_index          -0.264947
distance_id           -0.433405
run_up_distance        0.155734
purse                  0.014746
post_time             -0.078777
weight_carried        -0.292707
odds                   0.528807
ALW                   -0.296989
AOC                    0.287595
CLM                   -0.030051
SST                    0.269010
D                      0.238591
I                     -0.005133
M                     -0.326466
O                     -0.085747
T                      0.110222
FM                    -0.241833
FT                     0.165681
GD                     0.301027
MY                    -0.282943
SY                    -0.027239
AQU                    0.249910
BEL                   -0.171889
SAR                   -0.104090
position_at_finish     1.000000
Name: position_at_finish , dtype: float64

In [ ]:
X

,race_number,trakus_index,distance_id,run_up_distance,purse,post_time,weight_carried,odds,ALW,AOC,...,O,T,FM,FT,GD,MY,SY,AQU,BEL,SAR
0,9,73,600,48,25000,420,120,2090,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,9,74,600,48,25000,420,120,2090,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,9,63,600,48,25000,420,120,2090,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,9,64,600,48,25000,420,120,2090,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,9,65,600,48,25000,420,120,2090,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9,251,700,117,70000,522,114,1660,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9996,9,252,700,117,70000,522,114,1660,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9997,9,253,700,117,70000,522,114,1660,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9998,9,254,700,117,70000,522,114,1660,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
model=Sequential()
model.add(Dense(32,activation='elu', input_dim=25))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,batch_size=335,epochs=44,validation_data=(X_test,y_test))

Epoch 1/44
24/24 [==============================] - 2s 12ms/step - loss: 5855.8237 - accuracy: 0.0295 - val_loss: -6693.5732 - val_accuracy: 0.0880
Epoch 2/44
24/24 [==============================] - 0s 4ms/step - loss: -16064.8311 - accuracy: 0.0891 - val_loss: -26981.3203 - val_accuracy: 0.0880
Epoch 3/44
24/24 [==============================] - 0s 4ms/step - loss: -36563.3867 - accuracy: 0.0891 - val_loss: -48492.2734 - val_accuracy: 0.0880
Epoch 4/44
24/24 [==============================] - 0s 4ms/step - loss: -58632.0352 - accuracy: 0.0891 - val_loss: -71825.4688 - val_accuracy: 0.0880
Epoch 5/44
24/24 [==============================] - 0s 3ms/step - loss: -82553.8516 - accuracy: 0.0891 - val_loss: -97330.3203 - val_accuracy: 0.0880
Epoch 6/44
24/24 [==============================] - 0s 4ms/step - loss: -108750.8672 - accuracy: 0.0891 - val_loss: -125107.0234 - val_accuracy: 0.0880
Epoch 7/44
24/24 [==============================] - 0s 4ms/step - loss: -137397.1094 - accuracy: 0.0

In [ ]:
#How to select appropriate optimizers.
#!pip install -U keras-tuner

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(12))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 135 kB 6.9 MB/s 
     |████████████████████████████████| 1.6 MB 11.0 MB/s 


In [ ]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(X_train,y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 11s]
val_accuracy: 0.7099999785423279

Best val_accuracy So Far: 0.9543750286102295
Total elapsed time: 00h 02m 22s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 448 and the optimal learning rate for the optimizer
is 0.01.



In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train,y_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
200/200 [==============================] - 1s 4ms/step - loss: 1210.5750 - accuracy: 0.3698 - val_loss: 10.6897 - val_accuracy: 0.6300
Epoch 2/50
200/200 [==============================] - 1s 3ms/step - loss: 9.6978 - accuracy: 0.5427 - val_loss: 9.1007 - val_accuracy: 0.5806
Epoch 3/50
200/200 [==============================] - 1s 3ms/step - loss: 4.2920 - accuracy: 0.6200 - val_loss: 0.8378 - val_accuracy: 0.8069
Epoch 4/50
200/200 [==============================] - 1s 3ms/step - loss: 1.6464 - accuracy: 0.7472 - val_loss: 0.5871 - val_accuracy: 0.8363
Epoch 5/50
200/200 [==============================] - 1s 3ms/step - loss: 0.6221 - accuracy: 0.8480 - val_loss: 0.1506 - val_accuracy: 0.9162
Epoch 6/50
200/200 [==============================] - 1s 3ms/step - loss: 0.5066 - accuracy: 0.8655 - val_loss: 0.9791 - val_accuracy: 0.7669
Epoch 7/50
200/200 [==============================] - 1s 3ms/step - loss: 0.2797 - accuracy: 0.9125 - val_loss: 0.0836 - val_accuracy: 1.0000
Ep

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(X_train,y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/7
200/200 [==============================] - 1s 5ms/step - loss: 1292.2157 - accuracy: 0.3403 - val_loss: 18.4907 - val_accuracy: 0.3050
Epoch 2/7
200/200 [==============================] - 1s 3ms/step - loss: 4.6586 - accuracy: 0.4028 - val_loss: 1.4845 - val_accuracy: 0.4175
Epoch 3/7
200/200 [==============================] - 1s 3ms/step - loss: 1.5862 - accuracy: 0.4050 - val_loss: 1.4200 - val_accuracy: 0.4313
Epoch 4/7
200/200 [==============================] - 1s 3ms/step - loss: 1.4302 - accuracy: 0.4466 - val_loss: 1.3565 - val_accuracy: 0.4219
Epoch 5/7
200/200 [==============================] - 1s 3ms/step - loss: 1.3538 - accuracy: 0.4475 - val_loss: 1.3304 - val_accuracy: 0.4144
Epoch 6/7
200/200 [==============================] - 1s 3ms/step - loss: 1.4324 - accuracy: 0.4216 - val_loss: 1.5220 - val_accuracy: 0.4031
Epoch 7/7
200/200 [==============================] - 1s 3ms/step - loss: 1.4762 - accuracy: 0.4002 - val_loss: 1.4317 - val_accuracy: 0.4269
